In [1]:
from ipy_metadynamics import Metadynamics
import os
from oxdna_simulation import SimulationManager, Simulation, Observable
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

# plt.style.use(['science', 'no-latex', 'bright'])
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
path = os.path.abspath('/scratch/matthew/ipy_oxDNA/ipy_oxdna_examples/ico_3p/')
# systems = sorted(os.listdir(path))
systems = ['inital_umbrella']
file_dir_list = [f'{path}/{sys}/' for sys in systems]
sim_dir_list = [f'{file_dir}/meta_0' for sys, file_dir in zip(systems, file_dir_list)]
    
monomer_1_patch_1 = '11674,11666,11667,11668,11669,11670,11671,11672,11673'
monomer_1_patch_2 = '12945,12953,12946,12947,12948,12949,12950,12951,12952'
monomer_1_patch_3 = '13320,13328,13321,13322,13323,13324,13325,13326,13327'

monomer_2_patch_1 = '6655,6663,6656,6657,6658,6659,6660,6661,6662'
monomer_2_patch_2 = '6358,6350,6351,6352,6353,6354,6355,6356,6357'
monomer_2_patch_3 = '5752,5744,5745,5746,5747,5748,5749,5750,5751'

monomer_1 = f'{monomer_1_patch_1},{monomer_1_patch_2},{monomer_1_patch_3}'
monomer_2 = f'{monomer_2_patch_1},{monomer_2_patch_2},{monomer_2_patch_3}'

meta_list = [Metadynamics(file_dir, sim_dir) for file_dir, sim_dir in zip(file_dir_list, sim_dir_list)]
 
for meta in meta_list:
    meta.build(monomer_1, monomer_2,)

In [11]:
meta_arguments = [
    '--A', '50000', #Initial bias-height increment
    '--sigma', '5000', # Width of the deposited Gaussian
    '--tau', '10000', # Length of a metadynamics iteration (in time steps)
    '--dX', '0.001', # The spacing of the grid used to approximate the potential
    '--N_walkers', '8', # Number of parallel processes to be launched
    '--dT', '3', # Strength of the tempering (dT -> 0 leads to conventional simulations, dt -> infinity leads to conventional, non-well-tempered metadynamics)
    '--dim', '1', # Dimension of the order parameter
    '--p_fname', 'locs.meta', # File storing the indexes of the particles whose coordinates are used to build the order parameters
    '--ratio', '0', # Use the angle defined from the ratio of the distances between centres of mass as the order parameter
    '--angle', '0', # Use the angle defined from three centres of mass as the order parameter
    '--Niter', '100000', # Number of metadynamics iterations
    '--xmin', '0', # The lower boundary of the potential grid
    '--xmax', '20', # The upper boundary of the potential grid
    '--conf_interval', '100000', # Frequency with which configurations should be saved (in time steps)
    '--save_hills', '1', # Frequency with which the potential grid is sampled
    '--continue_run', '0', # Whether the simulation should continue or start anew
    '--T', '45C', #The temperature at which the simulations will be run. If not set, the temperature in the initial input file will be used
    '--use_sequential_GPUs' # Each walker will try to use a dedicated GPU: the first one will attempt to use GPU 1, the second one GPU 2, etc.
]

In [ ]:
invocation = meta_list[0].run(meta_arguments)

spawning
python3 /scratch/matthew/ipy_oxDNA/src/metad_interface.py /scratch/matthew/ipy_oxDNA/ipy_oxdna_examples/ico_3p/inital_umbrella/meta_0/base ['--A', '5000', '--sigma', '500', '--tau', '1000', '--dX', '0.001', '--N_walkers', '8', '--dT', '3', '--dim', '1', '--p_fname', 'locs.meta', '--ratio', '0', '--angle', '0', '--Niter', '100000', '--xmin', '0', '--xmax', '20', '--conf_interval', '100000', '--save_hills', '1', '--continue_run', '0', '--T', '45C', '--use_sequential_GPUs'] 

In [9]:
meta_list[0].meta_analysis.plot_distance(8)

EmptyDataError: No columns to parse from file

In [10]:
meta_list[-1].meta_analysis.plot_free_energy()

IndexError: index -1 is out of bounds for axis 0 with size 0

In [10]:
interval = list(range(int( len(meta_arguments))))[:-1:2]
meta_arguments_t = {meta_arguments[0 + i]:meta_arguments[1 + i] for i in interval}
if meta_arguments[-1] == '--use_sequential_GPUs':
    meta_arguments_t['--use_sequential_GPUs'] = True
    
meta_list[0].meta_arguments = meta_arguments_t